# Highway Vehicle Counting and Grouping


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 15.8 MB/s eta 0:00:00


#KMeans Model

In [2]:
import cv2
from ultralytics import YOLO

In [5]:
model = YOLO('/content/YOLO_Model.pt')

In [6]:
image_path = '/content/image.png'
image = cv2.imread(image_path)

In [7]:
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1.0
font_thickness = 2
bbox_thickness = 4

In [9]:
from sklearn.cluster import KMeans
import numpy as np
import random

# Number of clusters for KMeans
n_clusters = 4

# Apply YOLO model to the image
results = model.track(image, persist=True)

centroids = []
bbox_info = []

for result in results:
    for obj in result.boxes:
        bbox = obj.xyxy[0].cpu().numpy()  # Bounding box coordinates
        class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1  # Class ID
        conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0  # Confidence score
        id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1  # Unique ID

        # Convert bbox coordinates to integers
        x1, y1, x2, y2 = map(int, bbox)
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        centroids.append([cx, cy])
        bbox_info.append((x1, y1, x2, y2))  # Store bbox info

if centroids:
    # Apply KMeans clustering
    centroids = np.array(centroids)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(centroids)
    labels = kmeans.labels_

    # Generate unique colors for each cluster
    cluster_colors = {i: [random.randint(0, 255) for _ in range(3)] for i in range(n_clusters)}

    # Count the number of cars in each cluster
    cluster_counts = {i: 0 for i in range(n_clusters)}
    for label in labels:
        cluster_counts[label] += 1

    # Draw bounding boxes and cluster points with cluster-specific colors
    for idx, (x1, y1, x2, y2) in enumerate(bbox_info):
        cluster_label = labels[idx]
        color = cluster_colors[cluster_label]
        cv2.rectangle(image, (x1, y1), (x2, y2), color, bbox_thickness)
        cx, cy = centroids[idx]
        cv2.circle(image, (cx, cy), 5, color, -1)  # Mark cluster points

    # Display the cluster counts on the image
    y_offset = 50
    for label, count in cluster_counts.items():
        text = f'Cluster {label}: {count} cars'
        cv2.putText(image, text, (10, y_offset), font, font_scale, (0, 255, 255), font_thickness, cv2.LINE_AA)
        y_offset += 40



0: 480x640 44 cars, 4708.4ms
Speed: 4.9ms preprocess, 4708.4ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
# Save the output image with annotations
output_image_path = 'output_img_kmeans.jpg'
cv2.imwrite(output_image_path, image)

True

In [11]:
# Optionally, display the image (useful in local environments)
cv2.imshow('Image with KMeans Clustering', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
